In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
from itertools import chain
from sys import platform
from copy import deepcopy as dc
from scipy.interpolate import interp1d
from astropy.time import Time

vims_wave = np.loadtxt('vims_wave.txt')

new_array = ['_0405','_0607','_0809','_1011','_1213','_1415','_1617']
if platform == 'darwin' or platform == 'win32' or platform == 'linux':
    for number in new_array:
        new_string = 'df'+number+' = pd.read_pickle("total_data/data'+number+'.pkl")'
        exec(new_string)
else:
    df_list = []
    for number in new_array:
        new_string = 'df'+number+' = pd.read_csv("total_data/data'+number+'.csv")'
        exec(new_string)
        df_list.append(locals()['df'+number])
    for df in df_list:
        for i in range(len(df['spectrum'])):
            df['spectrum'][i] = np.array([float(x) for x in df['spectrum'][i][1:-1].split(',')])

vims_list = []
for i in range(4,18):
    if i<10:
        exec('vims_200'+str(i)+' = np.load("vims_shift/vims_200'+str(i)+'.npy")')
        vims_list.append(locals()['vims_200'+str(i)])
    else:
        exec('vims_20'+str(i)+' = np.load("vims_shift/vims_20'+str(i)+'.npy")')
        vims_list.append(locals()['vims_20'+str(i)])

def powerlaw(x,a,b):
    return a*np.power(x,b)

def gaussian(x,a,mu,sigma):
    return a*np.exp(-(x-mu)**2/(2*sigma**2))

def flatten(some_list):
    flat_list = []
    for element in some_list:
        if type(element) is list or type(element) is tuple:
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

band_channels = list(chain(range(29,35),range(46,60),range(78,96),range(102,106)))

window = list(range(56,83))

def fit_line(dataframe):
    spectra = np.average(dataframe)
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectra[band_channels],p0=[.15,-12])
    final_fit = powerlaw(vims_wave,*my_fit)
    return final_fit

def custom_fit(dataframe):
    spectra = np.average(dataframe)
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectra[band_channels],p0=[.15,-12])
    final_spectra = spectra-powerlaw(vims_wave,*my_fit)
    return final_spectra

def single_fit(spectrum):
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectrum[band_channels],p0=[.15,-12])
    final_spectrum = spectrum-powerlaw(vims_wave,*my_fit)
    return final_spectrum

def shift_ret(spectrum):
    my_fit,_ = curve_fit(gaussian,vims_wave[window],spectrum[window],p0=[.05,2.05])
    return my_fit[1]

In [2]:
scatter_list = []
for number in new_array:
    scatter_list.append(locals()['df'+number])

airmass_data = pd.concat(scatter_list,ignore_index=True)

airmass_data['emission'] *= (np.pi)/180
airmass_data['incidence'] *= (np.pi)/180

all_spectra = []
for i in range(len(airmass_data['spectrum'])):
    all_spectra.append(airmass_data['spectrum'][i])
all_spectra = np.asarray(all_spectra)

In [3]:
filter1 = dc(all_spectra)
filter2 = dc(all_spectra)
filter3 = dc(all_spectra)

bad_ind = np.where(filter1[:,69]<=.001)
filter1 = np.delete(filter1,bad_ind,axis=0)

p25 = np.percentile(filter2[:,69],25)
p75 = np.percentile(filter2[:,69],75)
inds = np.where((filter2[:,69] <= p25) | (filter2[:,69] >= p75))
filter2 = np.delete(filter2,inds,axis=0)

for i in range(256):
    filtered = False
    k = 0
    while filtered == False:
        mean_1 = np.mean(filter3[:,i])
        median = np.median(filter3[:,i])
        std = np.std(filter3[:,i])
        vals = filter3[:,i]
        ind = np.where(np.abs(vals-mean_1)>=3*std)
        filter3[ind,i] = mean_1+(3*std)
        mean_2 = np.mean(filter3[:,i])
        k += 1
        if np.abs(mean_1-mean_2) <= .05*mean_1:
            filtered = True

In [4]:
emission_vals = np.asarray(airmass_data['emission'])
incidence_vals = np.asarray(airmass_data['incidence'])
start_vals = np.asarray(airmass_data['starttime'])

channels = [
(9,20,37,65,107),
(12,24,43,70,115),
(16,27,47,76,118)
]

k_vals = [1.14,1.29,1.60,1.50,1.15]

channels = np.asarray(channels)

correction1 = dc(filter3)
correction2 = dc(filter3)
correction3 = dc(filter3)


aref = np.average(correction1[0,band_channels])
cor = (aref/(np.average(correction1[:,band_channels],axis=1))) # correction factor
correction1 *= cor[:,None]

correction2 /= np.cos(incidence_vals)[:,None]

for i in range(5):
    width = 1
    cor = ((k_vals[i]/2)*(correction3[:,channels[0,i]]+correction3[:,channels[2,i]]))
    correction3[:,channels[0,i]+width:channels[2,i]-width] -= cor[:,None]

In [5]:
shift_dates = [
'2004-01-01T00:00:00.00',
'2005-01-01T00:00:00.00',
'2005-07-01T00:00:00.00',
'2006-01-01T00:00:00.00',
'2006-07-01T00:00:00.00',
'2007-01-01T00:00:00.00',
'2007-07-01T00:00:00.00',
'2008-01-01T00:00:00.00',
'2008-07-01T00:00:00.00',
'2009-01-01T00:00:00.00',
'2009-07-01T00:00:00.00',
'2010-01-01T00:00:00.00',
'2010-07-01T00:00:00.00',
'2011-01-01T00:00:00.00',
'2011-07-01T00:00:00.00',
'2012-01-01T00:00:00.00',
'2012-07-01T00:00:00.00',
'2013-01-01T00:00:00.00',
'2013-07-01T00:00:00.00',
'2014-01-01T00:00:00.00',
'2014-04-19T00:00:00.00',
'2014-07-01T00:00:00.00',
'2015-01-01T00:00:00.00',
'2015-07-01T00:00:00.00',
'2016-01-01T00:00:00.00',
'2016-07-01T00:00:00.00',
'2017-01-01T00:00:00.00',
'2017-07-01T00:00:00.00',
'2017-10-19T00:00:00.00',
]

shift_vals = [0.0,0.0,0.4,1.1,1.9,2.6,3.4,4.1,4.9,5.6,6.0,6.0,6.0,6.0,7.0,7.3,7.7,8.0,8.4,8.7,9.1,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8]
shift_vals = np.asarray(shift_vals)
shift_vals /= 1000

date_vector = Time(shift_dates,format='isot').jd
date_interp = interp1d(date_vector,shift_vals)

all_shifts = date_interp(list(airmass_data['starttime']))

wave_shifts = []
year_list = []
for year in range(2004,2018):
    start_date = Time(str(year)+'-01-01T00:00:00.00',format='isot').jd
    end_date = Time(str(year+1)+'-01-01T00:00:00.00',format='isot').jd
    inds = np.where((start_vals >= start_date) & (start_vals <= end_date))[0]
    if len(inds) > 200:
        shift_ind = np.where(date_vector == Time(str(year)+'-07-01T00:00:00.00',format='isot').jd)[0][0] # Only using one value since all shifts should be the same for a specific date
        wave_shifts.append(shift_vals[shift_ind])
        year_list.append(year)

In [11]:
%matplotlib qt
fig,axs = plt.subplots([['a)','b)'],['c)','d)']],2,2,figsize=(30,22.5),facecolor='white')
for spectrum in filter3[::4]:
    axs[0,0].plot(vims_wave,spectrum)
    axs[0,0].grid()
    axs[0,0].set_title('Spectra Before Correction',fontsize=25,fontweight='bold')
    axs[0,0].set_xlabel('Wavelength (µm)',fontsize=18)
    axs[0,0].set_ylabel('I/F',fontsize=18)
for spectrum in correction1[::4]:
    axs[0,1].plot(vims_wave,spectrum)
    axs[0,1].grid()
    axs[0,1].set_title('Average Scaling Correction',fontsize=25,fontweight='bold')
    axs[0,1].set_xlabel('Wavelength (µm)',fontsize=18)
    axs[0,1].set_ylabel('I/F',fontsize=18)
for spectrum in correction2[::4]:
    axs[1,0].plot(vims_wave,spectrum)
    axs[1,0].grid()
    axs[1,0].set_title('Cosine Scaling Correction',fontsize=25,fontweight='bold')
    axs[1,0].set_xlabel('Wavelength (µm)',fontsize=18)
    axs[1,0].set_ylabel('I/F',fontsize=18)
for spectrum in correction3[::4]:
    axs[1,1].plot(vims_wave,spectrum)
    axs[1,1].grid()
    axs[1,1].set_title('K-Factor Window Correction',fontsize=25,fontweight='bold')
    axs[1,1].set_xlabel('Wavelength (µm)',fontsize=18)
    axs[1,1].set_ylabel('I/F',fontsize=18)
plt.show()

TypeError: subplots() takes from 0 to 2 positional arguments but 3 were given

In [16]:
%matplotlib qt
fig,axs = plt.subplots(2,2,facecolor='white',figsize=(30,22.5))
for spectrum in all_spectra[::3]:
    axs[0,0].plot(vims_wave,spectrum)
    axs[0,0].set_title('Original Data',fontsize=25,fontweight='bold')
    axs[0,0].set_xlabel('Wavelength (µm)',fontsize=18)
    axs[0,0].set_ylabel('I/F',fontsize=18)
for spectrum in filter1[::3]:
    axs[0,1].plot(vims_wave,spectrum)
    axs[0,1].set_title('Deletion Filter',fontsize=25,fontweight='bold')
    axs[0,1].set_xlabel('Wavelength (µm)',fontsize=18)
    axs[0,1].set_ylabel('I/F',fontsize=18)
for spectrum in filter2[::3]:
    axs[1,0].plot(vims_wave,spectrum)
    axs[1,0].set_title('Percentile Filter',fontsize=25,fontweight='bold')
    axs[1,0].set_xlabel('Wavelength (µm)',fontsize=18)
    axs[1,0].set_ylabel('I/F',fontsize=18)
for spectrum in filter3[::3]:
    axs[1,1].plot(vims_wave,spectrum)
    axs[1,1].set_title('Iterative Filter',fontsize=25,fontweight='bold')
    axs[1,1].set_xlabel('Wavelength (µm)',fontsize=18)
    axs[1,1].set_ylabel('I/F',fontsize=18)
plt.show()